In [1]:
import os
import pandas as pd
from tqdm import tqdm
import pickle

# Load talk data

In [2]:
talkfile = '../wikipedia/data/talk/enwiki/all_talk.csv'
talk = pd.read_csv(talkfile, header=None, names=['contribution_id', 'article', 'thread', 'timestamp', 'username', 'thread_type', 'text'])
talk

,contribution_id,article,thread,timestamp,username,thread_type,text
0,6356,Acharya S,POV: undue bias towards criticism,2010-11-21 06:44:51,^^James^^,THREAD_STARTER,http://en.wikipedia.org/wiki/Kenneth_Feder\nPr...
1,6358,Acharya S,POV: undue bias towards criticism,2010-11-21 15:36:50,^^James^^,POST,::You mean the assertion that Price is support...
2,51425,Da Drought 2,Children,2009-03-10 18:01:07,^-stephaanie,THREAD_STARTER,"Yeah man, Dwayne only has one daughter and one..."
3,119158,Lil Tunechi,Children,2009-03-10 18:01:07,^-stephaanie,THREAD_STARTER,"Yeah man, Dwayne only has one daughter and one..."
4,13301,Americans with Disabilities Act of 1990,Neutrality?,2005-03-10 16:06:30,^demon,POST,"In my personal opinion, this article is not ve..."
5,13324,Americans with Disabilities Act of 1990/Defini...,Neutrality?,2005-03-10 16:06:30,^demon,POST,"In my personal opinion, this article is not ve..."
6,82963,Google bomb,"Google search ""talentless hack"" rating",2006-12-20 02:23:06,^demon,THREAD_STARTER,The page claims that it itself is currently th...
7,145243,Montréal–Pierre Elliott Trudeau International ...,Controvercy,2007-03-25 11:09:38,^demonBot2,THREAD_STARTER,I work at YUL and there is and has been free w...
8,211468,Thule Air Base,troops,2007-03-25 11:07:46,^demonBot2,THREAD_STARTER,how many troops are stationed there
9,143799,Mobile PCI Express Module,Failure,2007-06-22 05:39:10,- - -,THREAD_STARTER,Nvidia just sell the chips for the MXM modules...


In [3]:
len(talk)

237207

# Filter threads

In [4]:
threads = set(zip(talk['article'], talk['thread']))
len(threads)

96533

In [5]:
# Load thread user counts
with open('../thread_usercounts.pkl', 'rb') as f:
    thread_num_users = pickle.load(f)

In [10]:
# Filter over n participants
threads_over = {}

for threshold in range(3,4):
    print(threshold)
    threads_over[threshold] = [name for name, count in thread_num_users.items() if count >= threshold]
    print(len(threads_over[threshold]))

    rows_over = {}
    mask = [t in threads_over[threshold] for t in zip(talk['article'], talk['thread'])]
    rows_over[threshold] = talk[mask]
    print(len(rows_over[threshold]))

    rows_over[threshold].sort_values(['article', 'contribution_id'], inplace=True)

    rows_over[threshold].to_csv('../wikipedia/data/talk/enwiki/talk_filtered_{0}.csv'.format(threshold), index=False)

3
15405
106399


/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [10]:
# Filter to threads with a certain number of participants--should pickle output
thread_num_users = {}
for a,t in tqdm(threads):
    rows = talk[(talk['article'] == a) & (talk['thread'] == t)]
    num_users = len(set(rows['username']))
    thread_num_users[(a,t)] = num_users

100%|██████████| 96533/96533 [51:42<00:00, 28.69it/s]


In [22]:
with open('../thread_usercounts.pkl', 'wb') as f:
    pickle.dump(thread_num_users, f)